![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [1]:
import pandas as pd
import numpy as np

# Start coding here...
df = pd.read_csv("bank_marketing.csv")

# Define columns for each DataFrame
client_columns = ['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']
campaign_columns = ['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']
economics_columns = ['client_id', 'cons_price_idx', 'euribor_three_months']

# Create each DataFrame by selecting the appropriate columns
client = df[client_columns]
campaign = df[campaign_columns]
economics = df[economics_columns]

# Assert that the data is subset correctly
assert set(client.columns) == set(client_columns), "Client DataFrame columns are incorrect."
assert set(campaign.columns) == set(campaign_columns), "Campaign DataFrame columns are incorrect."
assert set(economics.columns) == set(economics_columns), "Economics DataFrame columns are incorrect."

# Clean and format client.csv
client['job'] = client['job'].replace('.', '_', regex=True)
client['education'] = client['education'].replace('.', '_', regex=True)
client['education'] = client['education'].replace("unknown", np.NaN)

# Convert 'credit_default' to boolean: 1 if "yes", otherwise 0
client['credit_default'] = client['credit_default'].apply(lambda x: 1 if x == "yes" else 0).astype(bool)
assert client['credit_default'].dtype == 'bool', "credit_default column is not boolean."

# Convert 'mortgage' to boolean: 1 if "yes", otherwise 0
client['mortgage'] = client['mortgage'].apply(lambda x: 1 if x == "yes" else 0).astype(bool)
assert client['mortgage'].dtype == 'bool', "mortgage column is not boolean."

# Clean and format campaign.csv
campaign['previous_outcome'] = campaign['previous_outcome'].apply(lambda x: 1 if x == "yes" else 0).astype(bool)
campaign['campaign_outcome'] = campaign['campaign_outcome'].apply(lambda x: 1 if x == "yes" else 0).astype(bool)

# Check that the conversion was successful
assert campaign['previous_outcome'].dtype == 'bool', "previous_outcome column is not boolean."
assert campaign['campaign_outcome'].dtype == 'bool', "campaign_outcome column is not boolean."

# Define a mapping from month names to numeric values
month_mapping = {
    "jan": 1, "feb": 2, "mar": 3, "apr": 4, "may": 5, "jun": 6,
    "jul": 7, "aug": 8, "sep": 9, "oct": 10, "nov": 11, "dec": 12
}

# Map the month names to numeric values
df['month'] = df['month'].str.lower().map(month_mapping)
assert df['month'].notna().all(), "Month mapping failed for some rows."

# Add a new 'year' column with a fixed value of 2022
df['year'] = 2022

# Create the 'last_contact_date' column by combining 'year', 'month', and 'day'
df['last_contact_date'] = pd.to_datetime(df[['year', 'month', 'day']])

# Verify 'last_contact_date' data type
assert df['last_contact_date'].dtype == 'datetime64[ns]', "last_contact_date column is not datetime."

# Drop the 'year' column if it is no longer needed
df = df.drop(columns=['year'])

# Save each DataFrame as a separate CSV file without an index
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

# Verification of saved files
# Reload files and assert columns and data types
client_reloaded = pd.read_csv("client.csv")
campaign_reloaded = pd.read_csv("campaign.csv")
economics_reloaded = pd.read_csv("economics.csv")

assert set(client_reloaded.columns) == set(client_columns), "client.csv columns are incorrect after saving."
assert set(campaign_reloaded.columns) == set(campaign_columns), "campaign.csv columns are incorrect after saving."
assert set(economics_reloaded.columns) == set(economics_columns), "economics.csv columns are incorrect after saving."

print("All assertions passed successfully.")

All assertions passed successfully.
